#  Entity Explorer - Domain and URL
 <details>
     <summary> <u>Details...</u></summary>

 **Notebook Version:** 2.0<br>
 **Python Version:** Python 3.8 (including Python 3.8 - AzureML)<br>
 **Required Packages**: msticpy, msticnb<br>

 **Data Sources Required**:
 - Log Analytics - SecurityAlerts, Bookmarks, DnsEvents, CommonSecurityLog, DeviceNetworkEvents<br> 
**TI Proviers Used**
 - VirusTotal, Open Page Rank, BrowShot(all required for certain elements), AlienVault OTX, IBM XForce (optional) - all providers require accounts and API keys
 </details>

This Notebooks brings together a series of tools and techniques to enable threat hunting within the context of a domain name or URL that has been identified as of interest. It provides a series of techniques to assist in determining whether a domain or URL is malicious. Once this has been established it provides an overview of the scope of the domain or URL across an environment, along with indicators of areas for further investigation such as hosts of interest.  

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
<ul class="toc-item"><li><span><a href="#Hunting-Hypothesis:" data-toc-modified-id="Hunting-Hypothesis:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hunting Hypothesis:</a></span></ul>
<ul class="toc-item">
<li><span><a href="#Notebook-initialization" data-toc-modified-id="Notebook-initialization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Notebook initialization</a></span></li>
<ul class="toc-item"><li><span><a href="#Authentication" data-toc-modified-id="Authentication-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Authentication</a></span><ul class="toc-item"></li></ul>
<li><span><a href="#Import & Initalize Notebooklets" data-toc-modified-id="Import-Initialize-Notebooklets-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Import & Initalize Notebooklets</a></span></li></li>
<li><span><a href="#Select-the-domain-or-URL-you-wish-to-investigate" data-toc-modified-id="Select-the-domain-or-URL-you-wish-to-investigate-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Select the domain or URL you wish to investigate</a></span></li></ul>
<li><span><a href="#URL-Overview" data-toc-modified-id="URL-Overview-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>URL Overview</a></span>
<li><span><a href="#The-URL-in-the-environment" data-toc-modified-id="The-URL-in-the-environment-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>The URL in the Environmnet</a></span></li>
<ul class="toc-item"><li><span><a href="#Related-Alerts" data-toc-modified-id="Related-Alerts-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Related Alerts</a></span>
<li><span><a href="#Related-Bookmarks" data-toc-modified-id="Related-Bookmarks-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Related Bookmakrs</a></span></li>
<li><span><a href="#Hosts-Observed-Communicating-with-the-URL" data-toc-modified-id="Hosts-Observed-Communicating-with-the-URL-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Hosts Observed Communicating with the URL</a></span></li>
<li><span><a href="#Additional-Environment-Data" data-toc-modified-id="Additional-Environment-Data-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Additional Environment Data</a></span></li>
</ul></div>

## Hunting Hypothesis: 
Our broad initial hunting hypothesis is that a particular URL or domain might be malicious. <br>
This notebook is designed to help you explore the data and identify if the URL is malicious and where the URL appears within the environment.

---
### Notebook initialization
The next cell:
- Checks for the correct Python version
- Checks versions and optionally installs required packages
- Imports the required packages into the notebook
- Sets a number of configuration options.

<details>
    <summary><u>More details...</u></summary>

This should complete without errors. If you encounter errors or warnings look at the following two notebooks:
- [TroubleShootingNotebooks](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/TroubleShootingNotebooks.ipynb)
- [ConfiguringNotebookEnvironment](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)

If you are running in the Microsoft Sentinel Notebooks environment (Azure Notebooks or Azure ML) you can run live versions of these notebooks:
- [Run TroubleShootingNotebooks](./TroubleShootingNotebooks.ipynb)
- [Run ConfiguringNotebookEnvironment](./ConfiguringNotebookEnvironment.ipynb)

You may also need to do some additional configuration to successfully use functions such as Threat Intelligence service lookup and Geo IP lookup. 
There are more details about this in the `ConfiguringNotebookEnvironment` notebook and in these documents:
- [msticpy configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
- [Threat intelligence provider configuration](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html#configuration-file)

</details>


In [2]:
from IPython.display import display, HTML, Image
from datetime import datetime, timedelta, timezone


display(HTML("<h3>Starting Notebook setup...</h3>"))

# If not using Azure Notebooks, install msticpy and msticnb with
##%pip install msticpy --upgrade
##%pip install msticnb --upgrade

import msticpy as mp
extra_imports = [
    "msticpy.nbtools, observationlist",
    "msticpy.sectools, domain_utils",
    "pyvis.network, pyvis_network",
]
mp.init_notebook(
    extra_imports=extra_imports,
    additional_packages=["msticnb>=1.0"],
);

In [5]:
#%pip install pyvis.network
import pyvis_network
import msticpy as mp

In [6]:
# papermill default parameters
ws_name = "Default"
url = ""
end = datetime.now(timezone.utc)
start = end - timedelta(days=2)

### Authentication
<details>
    <summary><u>Details...</u></summary>
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. 
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. 
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

Note: you may occasionally see a JavaScript error displayed at the end of the authentication - you can safely ignore this.<br>
On successful authentication you should see a ```popup schema``` button.
To find your Workspace Id go to [Log Analytics](https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.
</details>

In [7]:
print(
    "Configured workspaces: ",
    ", ".join(msticpy.settings.get_config("AzureSentinel.Workspaces").keys()),
)
import ipywidgets as widgets

ws_param = widgets.Combobox(
    description="Workspace Name",
    value=ws_name,
    options=list(msticpy.settings.get_config("AzureSentinel.Workspaces").keys()),
)
ws_param

Configured workspaces:  Default


Combobox(value='Default', description='Workspace Name', options=('Default',))

In [8]:
from msticpy.common.timespan import TimeSpan

# Authentication
qry_prov = QueryProvider(data_environment="MSSentinel")
qry_prov.connect(WorkspaceConfig(workspace=ws_param.value))

nb_timespan = TimeSpan(start, end)
qry_prov.query_time.timespan = nb_timespan
md("<hr>")
md("Confirm time range to search", "bold")
qry_prov.query_time

try_azcli_login=True;enable_add_items_to_help=False
{'try_azcli_login': 'True', 'enable_add_items_to_help': 'False'}
try_azcli_login=True;enable_add_items_to_help=False
Connecting... 

connected


#### Authentication and Configuration Problems

<br>
<details>
    <summary>Click for details about configuring your authentication parameters</summary>
    
The notebook is expecting your Microsoft Sentinel Tenant ID and Workspace ID to be configured in one of the following places:
- `config.json` in the current folder
- `msticpyconfig.yaml` in the current folder or location specified by `MSTICPYCONFIG` environment variable.
    
For help with setting up your `config.json` file (if this hasn't been done automatically) see the [`ConfiguringNotebookEnvironment`](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb) notebook in the root folder of your Azure-Sentinel-Notebooks project. This shows you how to obtain your Workspace and Subscription IDs from the Microsoft Sentinel Portal. You can use the SubscriptionID to find your Tenant ID). To view the current `config.json` run the following in a code cell.

```%pfile config.json```

For help with setting up your `msticpyconfig.yaml` see the [Setup](#Setup) section at the end of this notebook and the [ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)
</details>

## Import and initialize notebooklets

This imports the **msticnb** package and the notebooklets classes.

These are needed for the notebook's operation.

In [9]:
import msticnb as nb

nb.init(query_provider=qry_prov)

Notebooklets: 11 notebooklets loaded.
try_azcli_login=True;enable_add_items_to_help=False
{'try_azcli_login': 'True', 'enable_add_items_to_help': 'False'}
try_azcli_login=True;enable_add_items_to_help=False
Notebooklets: Loaded providers: MSSentinel, geolitelookup, tilookup


## Select the domain or URL to investigate
Enter the domain or URL you wish to investigate. e.g. www.microsoft.com/index.html

In [47]:
#import ipywidgets as nbwidgets
#url_txt = nbwidgets.Text
#url_txt = nbwidgets.GetText(
#    prompt="Enter the URL to investigate:", value=url
#)
#display(url_txt)

url_txt = "https://chat.openai.com"

# URL Overview

The following cell runs the URL Summary notebooklet the collects relevant information about the URL, its domain, and associated IPs.<br>
Use the output to understand the context of the URL and where it appears in the environment. From here you can identify further areas and scopes to hunt on.

In [49]:
#print(url_txt)
url_nb = nb.nblts.azsent.url.URLSummary()
##url_test = nb.nblts.azsent.url.URLSummary
nb.init()
url_result = url_nb.run(value=url_txt, timespan=qry_prov.query_time.timespan)
print(url_result)


Notebooklets: 11 notebooklets loaded.
Notebooklets: Loaded providers: MSSentinel, geolitelookup, tilookup, azuredata


Observables processed:   0%|          | 0/2 [00:00<?, ?obs/s]

KeyError: None

In [44]:
nb.nblts.azsent.url.URLSummary(data_providers=nb.data_providers.DataProviders)

AttributeError: 'dict' object has no attribute 'DataProviders'

In [42]:
help(nb.nblts.azsent.url.URLSummary)

Help on class URLSummary in module msticnb.nb.azsent.url.url_summary:

class URLSummary(msticnb.notebooklet.Notebooklet)
 |  URLSummary(data_providers: Optional[<msticnb.data_providers.SingletonDecorator object at 0x0000018AD23596A0>] = None, **kwargs)
 |  
 |  URLSummary Notebooklet class.
 |  
 |  Queries and displays information about a URL including:
 |  
 |  - Overview of URL and Domain
 |  - Related alerts
 |  - TI results
 |  
 |  
 |  Default Options
 |  ---------------
 |  - ti: Displays TI results for the URL.
 |  - whois: Display a summary of the URL.
 |  - ip_record: Display a summary of the IP address the URL resolves to.
 |  - cert: Display a summary of TLS certs used by the URL.
 |  - alerts: Displays a DataFrame of all alerts associated with the URL.
 |  - bookmarks: Displays a DataFrame of all bookmarks associated with the URL.
 |  - dns: Displays a DataFrame of all DNS events associated with the URL.
 |  - hosts: Displays a DataFrame of all hosts associated with the U

In [39]:
#help(nb)
nb.init()

Notebooklets: 11 notebooklets loaded.
try_azcli_login=True;enable_add_items_to_help=False
{'try_azcli_login': 'True', 'enable_add_items_to_help': 'False'}
try_azcli_login=True;enable_add_items_to_help=False
Connecting... connected
try_azcli_login=True;enable_add_items_to_help=False
{'try_azcli_login': 'True', 'enable_add_items_to_help': 'False'}
try_azcli_login=True;enable_add_items_to_help=False
Notebooklets: Loaded providers: MSSentinel, geolitelookup, tilookup


# The URL in the Environment
Once we have determined the nature of the domain or URL under investigation we want to see what the scope of impact is in our environment but identifying any presence of the domain or URL in our datasets.<br>
If the domain has a high page rank score it is likely that it will be highly prevalent in a large environment, therefore you may wish to consider whether to run these cells for such a domain due to the data volumes involved.

In [33]:
if not url_result.dns_results.empty:
    url_result.dns_results.mp_plot.timeline(title=f"DNS lookup events for {url_txt.value}")
else:
    md(f"No dns events found relating to {url_txt.value}")

NameError: name 'url_result' is not defined

## Related Alerts
Understanding where a URL has appeared in alerts can help provide context on when a URL was first seen in an environment and its link to malicious activity.

In [34]:
url_result.display_alert_timeline()
url_result.browse_alerts()

NameError: name 'url_result' is not defined

## Related Bookmarks
As with alerts, understanding where a URL has appeared in bookmarks can help provide context on when a URL has previously been investigated within an environment.

In [ ]:
if not url_result.bookmarks.empty:
    url_result.bookmarks.mp_plot.timeline()
    display(url_result.bookmarks)
else:
    md(f"No bookmarks found relating to {url_txt.value}")

### Hosts Observed Communicating with the URL
During the cells executed above we have identified hosts communicating with the URL in question.  This cell provides a summary of these hosts. <br>
These hosts are potential candidates for further investigation using Microsoft Sentinel or via the host entity explorer Notebook.

In [ ]:
if url_result.hosts:
    md(f"{len(url_result.hosts)} hosts observed connecting with {url_txt.value}")
    for host in url_result.hosts:
        md(host)

## Additional environment data

To dig further into data regarding the URL in the environment there are a number of data sources returned by the URL Summary notebooklet. <br>
There are:<br>
- `dns_results` : shows DNS lookup events for the domain.
- `flows` : network flow logs for connections to the URL. 

They can be accessed by using `url_result.[data_source]` in the cells below.

---

## Use other notebooklets and pivots functions to drill down on other entities

You may want to drill down on other entities in the Host data.
You can use methods of the IpAddress or Host entities, for example,
to look at these in more detail.

Run the ip_address_summary notebooklet pivot
```python
IpAddress = entities.IpAddress
ip_result = IpAddress.nblt.ip_address_summary("157.56.162.53")
```

View the TI results
```python
ip_result.browse_ti_results()
```

---

# More information:

## Notebooklets and Pivots
[Notebooklets](https://msticnb.readthedocs.io/en/latest/)

[Pivot functions](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html)

## Notebook/MSTICPy configuration
[Getting Started](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A%20Getting%20Started%20Guide%20For%20Azure%20Sentinel%20ML%20Notebooks.ipynb)<br>
[MSTICPy Configuration guide](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)

[ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)